<a href="https://colab.research.google.com/github/LeeHyungDo/Mongotest/blob/master/HPC_SFT_%EC%8B%A4%EC%8A%B5%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 세팅

In [ ]:
%pip install -q transformers[torch] datasets
%pip install -q bitsandbytes trl peft
## 모델 인스트럭션 파인튜닝 학습 쉽게해주는 도구, 로라 데크닉 작은 파라미터만 학습, peft 로라위해서 쓴다.  도움주는 trl 라이브러리

## 데이터셋 불러오기


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("beomi/KoAlpaca-v1.1a")
## 한국어 지식 학습된 koalpaca db는 대부분 네이버 지식인 데이터 베이스로해서 chatgpt로 인스트럭션 셋 구성해달라고 해서 사용 self instruction

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

(…)-00000-of-00001-21df739eb88d711e.parquet:   0%|          | 0.00/12.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict

indices = range(0,1000)
test_indices = range(1000, 1050)
## lima 1000개 만 뽑아서 쓴다

dataset_dict = {"train": raw_datasets["train"].select(indices),
                "test": raw_datasets["train"].select(test_indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 50
    })
})

In [ ]:
def build_messages(example):
    messages = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["output"]}
    ]

    return dict(messages=messages)

raw_datasets = raw_datasets.map(build_messages)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url', 'messages'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['instruction', 'output', 'url', 'messages'],
        num_rows: 50
    })
})

In [ ]:
messages = raw_datasets["train"][0]["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))

user                :  양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
assistant           :  양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.


In [ ]:
example = raw_datasets["train"][0]
print(example.keys())

dict_keys(['instruction', 'output', 'url', 'messages'])


In this case, it looks like the instructions are about enabling certain features in Shopify. Interesting!

## 토크나이저 불러오기


In [ ]:
from transformers import AutoTokenizer

model_id = "Qwen/Qwen3-0.6B-Base"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

# Set chat template
# 메세지 포멧 만들었는데 구성 자동 갖추는게 템플릿, 각모델마다 다르다 똑같은 포멧하면 다른 결과, 확인은tokenizer_config.json에 나와있다.
# 실무는 lama나 chattemplate? 따르는 경우 많음
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## 챗 템플릿 적용하기

In [ ]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(), #map 병렬화가능
                                fn_kwargs={"tokenizer": tokenizer}, #두번째 파라미터 정의
                                remove_columns=column_names,
                                desc="Applying chat template",) #텍스트만 남겨두고 나머지 제거하기 위해

# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")

Applying chat template (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template (num_proc=8):   0%|          | 0/50 [00:00<?, ? examples/s]

Sample 601 of the processed training set:

<|system|>
<|endoftext|>
<|user|>
지구 둘레를 구하는 수학적 원리는 무엇인가요? 
지구 둘레를 구하는 기술적 원리와 그 원리를 통해 구한 값의 정확도에 대해 자세히 설명해주세요.<|endoftext|>
<|assistant|>
지구 둘레를 구하는 원리는 에라토스테네스가 발견했습니다. 그는 하짓날 정오에 시에네와 알렉산드리아에서 각각 햇빛이 비추는 각도를 측정하고, 이를 이용하여 지구의 둘레를 계산할 수 있습니다. 그의 가정은 다음과 같습니다. 

1. 알렉산드리아와 시에네(현재의 아스완 지역)사이의 거리는 5000 스타디아, 미터법으로 925km이다.
2. 지구는 완전한 구형으로 이루어져 있다.
3. 지구로 들어오는 햇빛은 평행하다.
4. 시에네는 북회선이 있어, 하짓날 정오에 햇빛이 지면에 수직으로 비춘다.
5. 알렉산드리아는 시에네의 정북쪽에 있다.

에라토스테네스는 하짓날 알렉산드리아에서 측정한 엇각을 이용하여 중심각의 크기를 측정했습니다. 이를 이용해 원에서 부채꼴의 중심각의 크기는 호의 길이와 정비례함을 이용하여 지구 둘레를 계산할 수 있습니다. 

그러나 지금은 이러한 방법으로 계산한 결과와 현재의 측정 값(40,030km)이 약간 차이가 있습니다. 이는 지구가 완전한 구형이 아니라 적도 부근이 조금 더 부푼 타원체이기 때문이고, 두 지점 사이의 거리를 구한 값에 인간의 측정 오차가 영향을 미친 것입니다. 

따라서, 에라토스테네스의 방법은 현재의 기술로는 완벽한 값이 아니지만, 그가 발견한 중심각의 원리는 지금까지도 사용되고 있으며, 측정의 정확도를 높이기 위해 여전히 연구되고 있습니다.<|endoftext|>

Sample 775 of the processed training set:

<|system|>
<|endoftext|>
<|user|>
왜 시력판은 2.0까지가 끝일까요? 옛날에는 2.0까지 보는 사람이 드물었지만, 요즘은 2.0까지 

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 50
    })
})

## 모델 학습 준비하기

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)
##로라 컨피그만 알고 있음 된다. r, lora_alpha값 신경, r랭크 파라미터 옆에 로라파라미터 사이즈 크면 클수록 옆에 붙이는 파라미터 커짐 적정숫자 필요, lora_alpha r에 비례하긴하는데 여기선 신경x
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)
# target_module 실무자 신경, 보통 이걿게 세팅, 모델 아키텍처마다 적용 모듈 찾는건 model 치면 이런 이름 embed_tokens나 q_proj, k_proj, v_proj 등에 나와있는 경우 붙인다. 모델 아키텍처마다 다르다 네이밍 구성 어떻게 해주는지 다를수 있음

model = get_peft_model(model, lora_config)
#위에만 해주면 로라 파라미터 적용됨, 전체 파라미터 몇개고 학습 파라미터 몇프로 학습되는지 나옴, 밑에 보면 2.98% 나옴

model.print_trainable_parameters()

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

trainable params: 18,350,080 || all params: 614,400,000 || trainable%: 2.9867


In [ ]:
from trl import SFTTrainer, SFTConfig
# trainer 에서 transfomer에서 받아왔는데 trl에서 받아와서 사용하면 된다.

output_dir = 'data/test_model'

# based on config
training_args = SFTConfig(
    fp16=True, ## A100 이상으로 가면 bp16
    do_eval=True,
    eval_strategy="epoch",
    gradient_accumulation_steps=8, ## batch 늘리면 이거 놀림 2*8 16으로 학습됨 여기는
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=5.0e-04,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine", # 학습할때 백프로파 할때 강도주는거 조정 learning rate 조정
    max_steps=-1,
    num_train_epochs=3,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=2, ## gpu한장에 학습할 배치 크기 몇개, 현재는 example 2개만 들어감
    save_strategy="epoch",
    save_total_limit=1,
    seed=42,## 체크
    dataset_text_field="text",
    packing=False, ##시퀀스 페킹 전체 예저 서로 다른데 덧붙여서 이어서 학습시킬지 선택하는거, true 로하면 맥락 겹침
    max_seq_length=tokenizer.model_max_length, ## 가장 중요 이게 길면 학습 시간 리소스 커진다 적정 길이 찾는거 중요
    report_to="tensorboard"
)

trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

Converting train dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/50 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## 학습해보기!

In [ ]:
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,000
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 186
  Number of trainable parameters = 18,350,080
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,1.968200,2.052452
2,1.862000,2.052175


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 50
  Batch size = 1
Saving model checkpoint to data/test_model/checkpoint-63
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B-Base/snapshots/11214f7f3465775dcce23c3752ecea5a42ee0ddc/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen3",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_k

In [ ]:
#응답 스타일 바꿔준다든지의 임팩트 lora 사용 좀더 많은 변화는 풀 파라 학습 필요

In [ ]:
trainer.model.save_pretrained(output_dir)
trainer.tokenizer.save_pretrained(output_dir)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B-Base/snapshots/11214f7f3465775dcce23c3752ecea5a42ee0ddc/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen3",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

Trainer.tokenizer is now deprecated. You should use Trainer.processing_c

('data/test_model/tokenizer_config.json',
 'data/test_model/special_tokens_map.json',
 'data/test_model/vocab.json',
 'data/test_model/merges.txt',
 'data/test_model/added_tokens.json',
 'data/test_model/tokenizer.json')

## 학습한 모델로 생성해보기

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#저장 모델 불러오기
output_dir = 'data/test_model'
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, device_map="auto")

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B-Base/snapshots/11214f7f3465775dcce23c3752ecea5a42ee0ddc/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen3",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "rms_norm_eps"

In [ ]:
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
import torch

messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": "HPC 이노베이션허브에 대해서 알고있어? 모르면 모른다고 이야기 해야해"},
]

## apply_chat_template로 질문할거 준비 add_generation_prompt는 어시스턴드 뒤에 토큰 만들어줌 천개 데이터 에폭 1000개 정도 시킨다 그래야 잘나온다함. lima에서
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.9,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<|system|>

<|user|>
HPC 이노베이션허브에 대해서 알고있어? 모르면 모른다고 이야기 해야해
<|assistant|>
HPC 이노베이션허브란 High Performance Computing(HPC) 기반 장비를 모아 사용하는 고성능 컴퓨팅 기반 업무를 할 수 있는 집합체를 말한다. 이론적으로는 HPC 가전과 데이터에 대해 대량의 처리를 할 수 있기 때문에 고성능 컴퓨팅 기반 업무를 할 수 있는 집합체로 기술적으로는 하이브리드 컴퓨팅의 구조다. 이에 따라 HPC 이노베이션허브란 하이브리드 컴퓨팅에서 처리량을 10배, 20배 높이는 데 이바로 적용된 컴퓨터를 의미한다. HPC 이노베이션허브란 1946년에 기존의 컴퓨터가 대대적으로 사용하는 방식과는 다른 방식으로 컴퓨터를 구동하기 때문에 HPC 이노베이션허브가 비지스터스 히트스텍으로, 대량의 처리로 향한 기술적 발전을 위해 도입된 장비다. 

추가 정보: 
- HPC 이노베이션허브를 가진
